In [144]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

📥 TMDB (The Movie Database) veri seti yükleniyor

In [86]:
movies=pd.read_csv('tmdb_5000_movies.csv')

👥 TMDB 'credits' veri seti yükleniyor

In [87]:
credits=pd.read_csv('tmdb_5000_credits.csv')

Movies dosyası her bir filmle ilgili başlık, açıklama, tür, popülerlik gibi bilgileri içerir

In [88]:
movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


Credits dosyasında her film için oyuncular (cast) ve ekip üyeleri (crew) gibi bilgiler bulunur

In [89]:
credits.head(5)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [90]:
movies.shape # Movies Boyutu

(4803, 20)

In [91]:
credits.shape # Credits Boyutu

(4803, 4)

🔗 'movies' ve 'credits' veri setlerini 'title' sütununa göre birleştiriyoruz

In [92]:
movies=movies.merge(credits,on='title')

Yeni boyut 4809 satır  23 sütun

In [93]:
movies.shape

(4809, 23)

Gereksiz sütunları atarak veri setini sadeleştiriyoruz

In [94]:
# Birleştirme
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]


🕵️‍♂️ Her sütundaki eksik (NaN) değerlerin sayısını kontrol ediyoruz

In [95]:
movies.isnull().sum()

,0
movie_id,0
title,0
overview,3
genres,0
keywords,0
cast,0
crew,0


🧼 Eksik (NaN) değer içeren satırları veri setinden tamamen kaldırıyoruz

In [96]:
movies.dropna(inplace=True)

🔍 Veri setinde tekrar eden (aynı) satırların sayısını kontrol ediyoruz

In [97]:
movies.duplicated().sum()

np.int64(0)

🔎 Veri setindeki ilk filmin (0. indeks) 'genres' (türler) bilgisini görüntülüyoruz

In [98]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

🧠 JSON benzeri bir string ifadeyi, Python'un anlayabileceği gerçek bir liste ve sözlük yapısına dönüştürüyoruz

In [99]:
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

JSON formatındaki string ifadeyi Python listesine çevirip,
içindeki her nesneden sadece 'name' değerlerini çıkaran fonksiyon

In [100]:
def convert(obj):
  L=[]
  for i in ast.literal_eval(obj):
        L.append(i['name'])
  return L

Convert Fonksiyonunu genres sütununda uygulama kısmı

In [101]:
movies['genres']=movies['genres'].apply(convert)

Genres(Tür) sütununun son hali

In [102]:
movies.head(5)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


İlk filmin (0. indeks) 'keywords' (anahtar kelimeler) sütunundaki veriyi görüntülüyoruz

In [103]:
movies.iloc[0].keywords

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

Convert Fonksiyonunu keywords sütununda sadece filmin anahtar kelimelerin name  alıyoruz id`leri atıyoruz

In [104]:
movies["keywords"]=movies["keywords"].apply(convert)

Keywords(Anahtar Kelimeler) sütununun son hali

In [105]:
movies.head(5)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


İlk filmin oyuncu kadrosu ('cast') bilgisini gösterir

In [106]:
movies.iloc[0].cast

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

 JSON formatındaki oyuncu listesinden sadece ilk 3 oyuncunun isimlerini alan fonksiyon

In [107]:
def convert2(obj):
   L=[]
   convert=0
   for i in ast.literal_eval(obj):
      if convert!=3:
        L.append(i['name'])
        convert+=1
      else:
        break
   return L


Convert2 Fonksiyonun  cast sütununa uygulayarak  filmlerdeki ilk 3 oyuncuyu alıyoruz

In [108]:
movies["cast"]=movies["cast"].apply(convert2)

Cast Sütunundan  ilk filmden alınan ilk 3 oyuncu

In [110]:
movies.iloc[0].cast

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']

Cast (Oyuncular) Sütununun  son hali

In [111]:
movies["cast"]

,cast
0,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]"
1,"[Johnny Depp, Orlando Bloom, Keira Knightley]"
2,"[Daniel Craig, Christoph Waltz, Léa Seydoux]"
3,"[Christian Bale, Michael Caine, Gary Oldman]"
4,"[Taylor Kitsch, Lynn Collins, Samantha Morton]"
...,...
4804,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua..."
4805,"[Edward Burns, Kerry Bishé, Marsha Dietlein]"
4806,"[Eric Mabius, Kristin Booth, Crystal Lowe]"
4807,"[Daniel Henney, Eliza Coupe, Bill Paxton]"


JSON formatındaki ekip listesinden 'Director' (yönetmen) olan kişinin adını bulan fonksiyon

In [112]:
# Yönetmeni yakalamak

def fetc_director(obj):
   L=[]
   for i in ast.literal_eval(obj):
      if i["job"]=="Director": # Mesleği yönetmen  olan kişiyi bul
          L.append(i["name"])  # Yönetmenin adını listeye ekler
          break                # İlk yönetmen bulunduğunda döngüyü bitir
   return L

Fetch director fonksiyonunun Crew (Ekip) sütununa uygulanması

In [113]:
movies["crew"]=movies["crew"].apply(fetc_director)

Crew (Ekip) sütununun son hali

In [114]:
movies["crew"]

,crew
0,[James Cameron]
1,[Gore Verbinski]
2,[Sam Mendes]
3,[Christopher Nolan]
4,[Andrew Stanton]
...,...
4804,[Robert Rodriguez]
4805,[Edward Burns]
4806,[Scott Smith]
4807,[Daniel Hsia]


İlk filmin (0. indeks) 'overview' (özet) metnini gösterir

In [115]:
movies["overview"][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

Tüm kelimeleri satır satır ayırdı

In [116]:
movies["overview"]=movies["overview"].apply(lambda x: x.split())

Eğer gelen veri bir string ise, içindeki tüm boşluk karakterlerini kaldırır
Değilse (örneğin NaN ya da başka bir tipse) olduğu gibi geri döner

In [117]:
# Tüm boşlukları silen fonksiyon
def delete_spaces(text):

    if isinstance(text, str):
        return text.replace(" ", "")
    return text


Boşluk varsa Sildik hepsini

In [118]:
movies['genres'] = movies['genres'].apply(delete_spaces)
movies['keywords'] = movies['keywords'].apply(delete_spaces)
movies['cast'] = movies['cast'].apply(delete_spaces)
movies['crew'] = movies['crew'].apply(delete_spaces)

"Overiew" + "Genres" +"Keywords" + "Cast" + "Crew" sütunlarını tag altında birleştirdik

In [119]:
 movies["tags"]=movies["overview"]+movies["genres"]+movies["keywords"]+movies["cast"]+movies["crew"]

In [120]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes],"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan],"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton],"[John, Carter, is, a, war-weary,, former, mili..."


 'movies' veri setinden sadece 'movie_id', 'title' ve 'tags' sütunlarını seçip yeni bir DataFrame oluşturuyoruz

In [121]:
new_data=movies[["movie_id","title","tags"]]

Yeni DataFrame Özellikleri

In [122]:
new_data.head()

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


'tags' sütunundaki liste halindeki kelimeleri aralarına boşluk koyarak tek bir stringe dönüştürüyoruz Böylece kelimeler arasındaki virgüller kaldırılmış oluyor

In [123]:
new_data["tags"]=new_data["tags"].apply(lambda x:" ".join(x))

<ipython-input-123-64b032f56e7c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["tags"]=new_data["tags"].apply(lambda x:" ".join(x))


Tags bütün kelimeleri küçük yaptık

In [124]:
new_data["tags"]=new_data["tags"].apply(lambda x:x.lower())

<ipython-input-124-a8964160f85f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["tags"]=new_data["tags"].apply(lambda x:x.lower())


In [125]:
new_data

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,a newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...


Metin verisini sayısal vektörlere dönüştürmek için CountVectorizer kullanıyoruz
En fazla 5000 kelimeyi alır, İngilizce'deki anlamsız (stop) kelimeleri yok sayar

In [126]:
model=CountVectorizer(max_features=5000,stop_words='english')

'tags' sütunundaki metinleri sayısal vektörlere dönüştürüp dizi (array)formatına çeviriyoruz

In [127]:
vector=model.fit_transform(new_data["tags"]).toarray() #

In [128]:
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

CountVectorizer tarafından çıkarılan tüm kelime (özellik) listesini verir

In [129]:
model.get_feature_names_out() # 5000 kelimelik sözlüğü getirir.

array(['000', '007', '10', ..., 'zooey', 'zucker', 'zwick'], dtype=object)

Kelimeleri köklerine indirgemek (stem) için PorterStemmer kullanıyoruz

In [130]:
ps=PorterStemmer()

In [131]:
def stem(text): # Köke indirger
  y=[]
  for i in text.split(): # Metni kelimelere ayırır
    y.append(ps.stem(i)) # Kelime kökünü alıp listeye ekler
  return " ".join(y)  # en son tekrar birleştirir

Kelimenini köküne indirme örneği

In [132]:
ps.stem('loved')

'love'

Tüm 'tags' verilerini köklerine indiriyoruz (stem işlemi uyguluyoruz)

In [133]:
new_data["tags"]=new_data["tags"].apply(stem)

<ipython-input-133-bc497b5cc05b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["tags"]=new_data["tags"].apply(stem)


Filmler arasındaki benzerlikleri kosinüs benzerliği ile hesaplıyoruz

In [134]:
similarity=cosine_similarity(vector)
similarity[3][10]# = 0.24 → Film 4 ve Film 11 %5.9 benzer.

np.float64(0.059131239598908265)

In [135]:
similarity[0] # Benzerlikleri 1 olan kendisi

array([1.        , 0.06885304, 0.04948717, ..., 0.03142697, 0.05345225,
       0.        ])

In [136]:
similarity[0][3221] # 1film ile 3221 film 0.201 benzer

np.float64(0.02010075630518424)

 similarity matrisinin boyutunu gösterir (film sayısı x film sayısı) 4806,4806

In [137]:
similarity.shape

(4806, 4806)

Verilen bir film adına göre, benzerlik skorlarına dayanarak en çok benzeyen 5 filmi önerir ve isimlerini yazdırır.

In [138]:
def recommend(movie):
  movie_index=new_data[new_data["title"]==movie].index[0]
  distances=similarity[movie_index]
  movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6] #En yüksek benzerlik skorlarına göre ilk 5 filmi seçer (kendisi hariç)


  for i in movies_list: #Seçilen benzer filmlerin indekslerine göre isimlerini yazdırır
    print(new_data.iloc[i[0]].title)


"Silent Running" filminin new_data DataFrame'indeki indeks numarasını bulur

In [139]:
new_data[new_data["title"]=="Silent Running"].index[0]

np.int64(4338)

Bu filmin diğer tüm filmlerle benzerlik skorlarını alır

In [140]:
distances=similarity[2] # 0 filmin kendisiyle benzerlikleri
print(distances)

[0.04948717 0.04259177 1.         ... 0.01944039 0.08817334 0.        ]


 1. ile 3. filmin benzerlikleri yazar

In [141]:
distances=similarity[0][2]
print(distances)

0.04948716593053934


Büyükten küçüğe sıralar  kendisine benzeyen filmleri

In [142]:
distances=similarity[3]
distances=sorted(distances,reverse=True)
print(distances)

[np.float64(1.0000000000000004), np.float64(0.5095246653650681), np.float64(0.45798926118677813), np.float64(0.4202488610103944), np.float64(0.4147939365757017), np.float64(0.39234898870486334), np.float64(0.34869483697070264), np.float64(0.34847009003341817), np.float64(0.29417420270727607), np.float64(0.28412058716342514), np.float64(0.24514516892273006), np.float64(0.24255538756437856), np.float64(0.23671603823699966), np.float64(0.22895273494481277), np.float64(0.2207714951038695), np.float64(0.21572774865200245), np.float64(0.2139802462554565), np.float64(0.2135311066302971), np.float64(0.20830127958541944), np.float64(0.20464687117164013), np.float64(0.20155644370746376), np.float64(0.2002453486917181), np.float64(0.20000907091185807), np.float64(0.1985831017383096), np.float64(0.19858310173830954), np.float64(0.19439829648023824), np.float64(0.1934477640657483), np.float64(0.19088542889273336), np.float64(0.19037490262714746), np.float64(0.19037490262714746), np.float64(0.186816

Tahmin

In [143]:
recommend("Avatar")

Aliens
Moonraker
Alien
Alien³
Silent Running


Film verilerini sözlük olarak .pkl dosyasına kaydeder

In [ ]:
pickle.dump(new_data.to_dict(),open("movie_dict.pkl","wb"))

 Benzerlik matrisini .pkl dosyasına kaydeder

In [ ]:
pickle.dump(similarity,open("similarity.pkl","wb"))